### Visualizing Finite State Automata

Consider the following declarations from the course notes:

In [1]:
class set(frozenset):
    def __repr__(self):
        return '{' + ', '.join(str(e) for e in self) + '}'

class FiniteStateAutomaton:
    def __init__(self, T, Q, R, q0, F):
        self.T, self.Q, self.R, self.q0, self.F = T, Q, R, q0, F
    def __repr__(self):
        return str(self.q0) + '\n' + ' '.join(str(f) for f in self.F) + '\n' + \
               '\n'.join(str(q) + ' ' + a + ' → ' + str(r) for (q, a, r) in self.R)

def parseFSA(fsa: str) -> FiniteStateAutomaton:
    fsa = [line for line in fsa.split('\n') if line.strip() != '']
    q0 = fsa[0] # first line: initial
    F = set(fsa[1].split()) # second line: final, final, ...
    R = set()
    for line in fsa[2:]: # all subsequent lines: "source symbol → target"
        l, r = line.split('→')
        R |= {(l.split()[0], l.split()[1], r.split()[0])}
    T = {r[1] for r in R}
    Q = {q0} | F | {r[0] for r in R} | {r[2] for r in R}
    return FiniteStateAutomaton(T, Q, R, q0, F)

def minimizeFSA(fsa: FiniteStateAutomaton) -> FiniteStateAutomaton:
    δ = {(q, a): r for (q, a, r) in fsa.R}
    dist = {(q, r) for q in fsa.Q for r in fsa.Q if q != r and (q in fsa.F) != (r in fsa.F)}
    done = False
    while not done:
        done = True #; print(dist)
        for q in fsa.Q:
            for r in fsa.Q:
                if q != r and (q, r) not in dist and any(((q, u) in δ) != ((r, u) in δ) or \
                    ((q, u) in δ) and ((δ[(q, u)], δ[(r, u)]) in dist) for u in fsa.T):
                    dist |= {(q, r)}; done = False #; print('adding', q, r)
    Qʹ = {set({q} | {r for r in fsa.Q if (q, r) not in dist}) for q in fsa.Q}
    Rʹ = {(qʹ, u, rʹ) for qʹ in Qʹ for rʹ in Qʹ for u in fsa.T if any((q, u, r) in fsa.R for q in qʹ for r in rʹ)}
    qʹ0 = {qʹ for qʹ in Qʹ if fsa.q0 in qʹ}.pop()
    Fʹ = {qʹ for qʹ in Qʹ if (qʹ & fsa.F) != set()}
    return FiniteStateAutomaton(fsa.T, Qʹ, Rʹ, qʹ0, Fʹ)

def totalFSA(A: FiniteStateAutomaton, t = -1) -> FiniteStateAutomaton:
    T = set('abcdefghijklmnopqrstuvwxyz') # T is vocabulary, t is trap state
    R = A.R | {(q, a, t) for q in A.Q for a in T if all((q, a, r) not in A.R for r in A.Q)}
    if any(r == t for (q, a, r) in R): # transition to t exists
        Q = A.Q | {t}
        R = R | {(t, a, t) for a in T}
    else: Q = A.Q
    return FiniteStateAutomaton(T, Q, R, A.q0, A.F)

def renameFSA(fsa: FiniteStateAutomaton) -> FiniteStateAutomaton:
    m, c = {}, 0
    for q in fsa.Q:
        m[q] = c; c = c + 1
    Qʹ = {i for i in range(c)}
    Rʹ = {(m[q], u, m[r]) for (q, u, r) in fsa.R}
    qʹ0 = m[fsa.q0]
    Fʹ = {m[q] for q in fsa.F}
    return FiniteStateAutomaton(fsa.T, Qʹ, Rʹ, qʹ0, Fʹ)

def equivalentFSA(a: FiniteStateAutomaton, aʹ: FiniteStateAutomaton, printMap = False) -> bool:
    a = totalFSA(minimizeFSA(a))
    aʹ = totalFSA(minimizeFSA(aʹ))
    δ = {(q, u): r for (q, u, r) in a.R}
    δʹ = {(q, u): r for (q, u, r) in aʹ.R}
    m, v = {a.q0: aʹ.q0}, {a.q0}
    while v != set():
        q = v.pop(); qʹ = m[q]
        for u in a.T:
            r, rʹ = δ[(q, u)], δʹ[(qʹ, u)]
            if r in m:
                if m[r] != rʹ: return False
            elif rʹ in m.values(): return False
            else: v.add(r); m[r] = rʹ
    if printMap: print(m) 
    return aʹ.F == {m[q] for q in a.F}

Consider the two deterministic finite state automata below. 

In [2]:
A0 = parseFSA("""
0
0
0 a → 1
0 b → 2
1 b → 0
2 a → 0
2 c → 3
3 a → 3
""")
A1 = parseFSA("""
x
x
x b → z
y b → x
z a → x
x a → y
w a → w
""")
A0, A1

(0
 0
 3 a → 3
 0 b → 2
 2 c → 3
 0 a → 1
 1 b → 0
 2 a → 0,
 x
 x
 x a → y
 x b → z
 z a → x
 w a → w
 y b → x)

1. Draw the finite state diagrams of `A0` and `A1`!
2. Draw the finite state diagrams of the equivalent but total automata, i.e. of `A0tot = totalFSA(A0)` and `A1tot = totalFSA(A1)`!
3. Draw the finite state diagrams of the equivalent, total, and minimized automata, i.e. `A0min = minimizeFSA(A0tot)` and `A1min = minimizeFSA(A1tot)`!
4. Draw the finite state diagrams of the equivalent, total, minimized, and renamed automata, i.e. of `A0opt = renameFSA(A0min)` and `A1opt = renameFSA(A1min)`!

You may use https://finsm.io/.

Your solution here

_Instructor's Solution:_

1. `A0`: <img src="img/A0.svg"/>  `A1`: <img src="img/A1.svg"/>

2. `A0tot`: <img src="img/A0tot.svg"/>  `A1tot`: <img src="img/A1tot.svg"/>

In [ ]:
A0tot, A1tot = totalFSA(A0), totalFSA(A1); A0tot, A1tot

3. `A0min`: <img src="img/A0min.svg"/>  `A1min`: <img src="img/A1min.svg"/>  

The Python function `minimizeFSA` iterates over sets in a nondeterministic order. There is no guarantee that when running the function, the result will be the same as in the diagrams, although the result will be equivalent.

In [ ]:
A0min, A1min = minimizeFSA(A0tot), minimizeFSA(A1tot); A0min, A1min

4. `A0opt`: <img src="img/A0opt.svg"/>  `A1min`: <img src="img/A1opt.svg"/>  

The Python function `renameFSA` iterates over sets in nondeterministic order. There is no guarantee that when running the function, the result will be the same as in the diagrams, although the result will be equivalent.

In [ ]:
A0opt, A1opt = renameFSA(A0min), renameFSA(A1min); A0opt, A1opt

The algorithm for checking the equivalence of two deterministic and minimal finite state automata, as implemented by `equivalentFSA` above, iteratively constructs a bijection from states of one automaton to states of the other automaton. Construct this bijection manually for `A0` and `A1` above.

Your solution here

*Instructor's Solution:*

In the implementation above, this bijection is implemented by the Python dictionary `m`. The answer can be obtained by setting `printMap` to `True`:

In [ ]:
assert equivalentFSA(A0opt, A1opt, True)

What is the relation between the dictionary that `equivalentFSA(A0, A1)` constructs and that `equivalentFSA(A1, A0)` constructs, for arbitrary `A0`, `A1`? Check your answer by running `equivalentFSA(A0opt, A1opt, True)` and `equivalentFSA(A1opt, A0opt, True)`!

Your solution here

*Instructor's Solution:*

As the constructed dictionary is a bijection, those of `equivalentFSM(A0, A1)` and `equivalentFSM(A1, A0)` are inverses of each other:

In [ ]:
assert equivalentFSA(A1opt, A0opt, True)